In [1]:
import sys
sys.path.append('..')
import os
import sklearn.metrics.cluster
import sklearn.cluster
import seaborn as sns
import scanpy as sc
import sklearn as sk
from sklearn.neighbors import NearestNeighbors
import matplotlib as mpl
from matplotlib import pyplot as plt
import skdim
import seaborn as sns
import time
import numpy as np
import pandas as pd
import pickle
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages
import utils
import anndata2ri
import gc
import benchmark_common_functions
from itertools import product

anndata2ri.activate()
ro.numpy2ri.activate()
readRDS = ro.r['readRDS']
%load_ext rpy2.ipython

# Run raw datasets

In [2]:
rootdir = '../data/Freytag_datasets/'
exclude=['sce_full_PBMC3k1.rds', 'sce_full_PBMC3k2.rds', 'sce_full_PBMC4k1.rds', 'sce_full_PBMC4k2.rds']
fnames = os.listdir(rootdir)
fnames = [f for f in fnames if f not in exclude and 'labels' not in f]

resdir = '../results/Freytag_datasets/'

#get_res_path_raw = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'
get_res_path_raw = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'
get_res_path_raw_sc = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}_scanpyhubness.pkl'

In [3]:
#fixed params
do_log = True
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('duo','seurat',)#,'zheng17')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))

In [4]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path_raw(fname).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname.split('/')[-1])
        
        if 'PBMC' in fname:
            adata = sc.read(rootdir+fname).T
            adata.obs['phenoid'] = pd.read_csv(rootdir+fname[:-4]+'_labels.csv')['x'].values        
        else:
            adata = readRDS(rootdir+fname)
            
        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['phenoid'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            n_clusters= adata.obs['phenoid'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_raw(fname.split('/')[-1]),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True duo True
cosine 25 True duo False
cosine 25 True seurat True
cosine 25 True seurat False
cosine 50 True duo True
cosine 50 True duo False
cosine 50 True seurat True
cosine 50 True seurat False
cosine 100 True duo True
cosine 100 True duo False
cosine 100 True seurat True
cosine 100 True seurat False
cosine 500 True duo True
cosine 500 True duo False
cosine 500 True seurat True
cosine 500 True seurat False
euclidean 25 True duo True
euclidean 25 True duo False
euclidean 25 True seurat True
euclidean 25 True seurat False
euclidean 50 True duo True
euclidean 50 True duo False
euclidean 50 True seurat True
euclidean 50 True seurat False
euclidean 100 True duo True
euclidean 100 True duo False
euclidean 100 True seurat True
euclidean 100 True seurat False
euclidean 500 True duo True
euclidean 500 True duo False
euclidean 500 True seurat True
euclidean 500 True seurat False


In [4]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path_raw_sc(fname).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname.split('/')[-1])
        
        if 'PBMC' in fname:
            adata = sc.read(rootdir+fname).T
            adata.obs['phenoid'] = pd.read_csv(rootdir+fname[:-4]+'_labels.csv')['x'].values        
        else:
            adata = readRDS(rootdir+fname)
            
        results_dict = benchmark_common_functions.scanpy_hubness_analysis(
            adata,
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_raw_sc(fname.split('/')[-1]),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True duo True
		sce_full_ChuBatch1.rds


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		duo norm retained 5000 genes
			Scoring: 0.41 mn
		sce_full_ChuBatch2.rds


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		duo norm retained 5000 genes
			Scoring: 0.12 mn
		sce_full_FreytagGold.rds


...

		seurat norm retained 5448 genes
			Scoring: 0.03 mn
		sce_full_ChuBatch2.rds


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		seurat norm retained 5445 genes
			Scoring: 0.06 mn
		sce_full_FreytagGold.rds


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		seurat norm retained 4033 genes
			Scoring: 0.24 mn
		sce_full_PBMC3k1.csv
		seurat norm retained 2404 genes
			Scoring: 0.4 mn
		sce_full_PBMC3k2.csv
		seurat norm retained 1852 genes
			Scoring: 0.54 mn
		sce_full_PBMC4k1.csv
		seurat norm retained 2234 genes
			Scoring: 0.8 mn
		sce_full_PBMC4k2.csv
		seurat norm retained 1757 genes
			Scoring: 0.43 mn
